In [39]:
# import all necessary library
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
import io
import pandas as pd
import numpy as np
#import osmnx as ox

In [40]:
# Dictionary mapping abbreviations to full county names
county_full_names = {
    'ALA': 'Alameda',
    'ALP': 'Alpine',
    'AMA': 'Amador',
    'BUT': 'Butte',
    'CAL': 'Calaveras',
    'COL': 'Colusa',
    'CC': 'Contra Costa',
    'DN': 'Del Norte',
    'ED': 'El Dorado',
    'FRE': 'Fresno',
    'GLE': 'Glenn',
    'HUM': 'Humboldt',
    'IMP': 'Imperial',
    'INY': 'Inyo',
    'KER': 'Kern',
    'KIN': 'Kings',
    'LAK': 'Lake',
    'LAS': 'Lassen',
    'LA': 'Los Angeles',
    'MAD': 'Madera',
    'MRN': 'Marin',
    'MPA': 'Mariposa',
    'MEN': 'Mendocino',
    'MER': 'Merced',
    'MOD': 'Modoc',
    'MNO': 'Mono',
    'MON': 'Monterey',
    'NAP': 'Napa',
    'NEV': 'Nevada',
    'ORA': 'Orange',
    'PLA': 'Placer',
    'PLU': 'Plumas',
    'RIV': 'Riverside',
    'SAC': 'Sacramento',
    'SBT': 'San Benito',
    'SBD': 'San Bernardino',
    'SD': 'San Diego',
    'SF': 'San Francisco',
    'SJ': 'San Joaquin',
    'SLO': 'San Luis Obispo',
    'SM': 'San Mateo',
    'SB': 'Santa Barbara',
    'SCL': 'Santa Clara',
    'SCR': 'Santa Cruz',
    'SHA': 'Shasta',
    'SIE': 'Sierra',
    'SIS': 'Siskiyou',
    'SOL': 'Solano',
    'SON': 'Sonoma',
    'STA': 'Stanislaus',
    'SUT': 'Sutter',
    'TEH': 'Tehama',
    'TRI': 'Trinity',
    'TUL': 'Tulare',
    'TUO': 'Tuolumne',
    'VEN': 'Ventura',
    'YOL': 'Yolo',
    'YUB': 'Yuba'
}

In [41]:
# read state highway system
shs=gpd.read_file('/mnt/mdp/Jupyter/Dev_Mintu/PeMS_Coverage/SHN_Lines.geojson')
shs_proj=shs.to_crs("EPSG:3310")
shs_proj['County'] = shs_proj['County'].map(county_full_names)
shs_proj['County']=shs_proj['County'].str.lower()
shs_proj.head()

,OBJECTID,Route,RteSuffix,RouteS,PMRouteID,County,District,PMPrefix,bPM,ePM,PMSuffix,bPMc,ePMc,bOdometer,eOdometer,AlignCode,RouteType,Direction,Shape_Length,geometry
0,1,80,,80,PLA080.R.L,placer,3,R,55.008,56.078,,R55.008,R56.078,156.512,157.582,Left,Interstate,WB,0.018392,"MULTILINESTRING ((-58329.675 142277.358, -5832..."
1,2,80,,80,PLA080...L,placer,3,,52.947,55.008,,52.947,55.008,154.451,156.512,Left,Interstate,WB,0.036842,"MULTILINESTRING ((-61268.303 140813.005, -6126..."
2,3,80,,80,PLA080..LL,placer,3,,49.180,52.947,L,49.18L,52.947L,150.684,154.451,Left Independent,Interstate,WB,0.060406,"MULTILINESTRING ((-64901.249 136251.945, -6490..."
3,4,80,,80,PLA080...L,placer,3,,26.920,49.180,,26.92,49.18,128.424,150.684,Left,Interstate,WB,0.362514,"MULTILINESTRING ((-85592.441 111385.108, -8558..."
4,5,80,,80,PLA080...L,placer,3,,0.000,18.822,,0.0,18.822,101.519,120.341,Left,Interstate,WB,0.310451,"MULTILINESTRING ((-112373.646 79179.862, -1122..."


In [42]:
shs_proj_link=shs.to_crs("EPSG:3310")
shs_proj_link['Segment_length_mile']=shs_proj_link['geometry'].length*0.000621371
shs_proj_link.head()

,OBJECTID,Route,RteSuffix,RouteS,PMRouteID,County,District,PMPrefix,bPM,ePM,...,bPMc,ePMc,bOdometer,eOdometer,AlignCode,RouteType,Direction,Shape_Length,geometry,Segment_length_mile
0,1,80,,80,PLA080.R.L,PLA,3,R,55.008,56.078,...,R55.008,R56.078,156.512,157.582,Left,Interstate,WB,0.018392,"MULTILINESTRING ((-58329.675 142277.358, -5832...",1.053197
1,2,80,,80,PLA080...L,PLA,3,,52.947,55.008,...,52.947,55.008,154.451,156.512,Left,Interstate,WB,0.036842,"MULTILINESTRING ((-61268.303 140813.005, -6126...",2.067330
2,3,80,,80,PLA080..LL,PLA,3,,49.180,52.947,...,49.18L,52.947L,150.684,154.451,Left Independent,Interstate,WB,0.060406,"MULTILINESTRING ((-64901.249 136251.945, -6490...",3.723561
3,4,80,,80,PLA080...L,PLA,3,,26.920,49.180,...,26.92,49.18,128.424,150.684,Left,Interstate,WB,0.362514,"MULTILINESTRING ((-85592.441 111385.108, -8558...",22.255459
4,5,80,,80,PLA080...L,PLA,3,,0.000,18.822,...,0.0,18.822,101.519,120.341,Left,Interstate,WB,0.310451,"MULTILINESTRING ((-112373.646 79179.862, -1122...",18.833949


In [43]:
# import PeMS meta data
pems_meta_data=pd.read_csv('/mnt/mdp/Jupyter/Dev_Mintu/PeMS_Coverage/detector_meta_data.csv')
pems_meta_data=pems_meta_data.loc[pems_meta_data['STATUS']==1]
pems_meta_data1=pems_meta_data.drop_duplicates(subset=['STATION_ID'], keep='first')
pems_meta_data2=pems_meta_data1.drop(['GEOMETRY'],axis=1)
geometry = [Point(xy) for xy in zip(pems_meta_data2['LONGITUDE'], pems_meta_data2['LATITUDE'])]
pems_gdf = gpd.GeoDataFrame(pems_meta_data2, geometry=geometry)

# Set the CRS to EPSG:4326 (WGS 84)
pems_gdf.set_crs(epsg=4326, inplace=True)
pems_gdf_proj=pems_gdf.to_crs("EPSG:3310")
pems_gdf_proj = pems_gdf_proj[(pems_gdf_proj['STATION_TYPE'] == 'ML') | (pems_gdf_proj['STATION_TYPE'] == 'HV')]
# Replace direction codes in the 'Direction' column
pems_gdf_proj['DIRECTION'] = pems_gdf_proj['DIRECTION'].replace({
    'N': 'NB',
    'S': 'SB',
    'W': 'WB',
    'E': 'EB'
})
pems_gdf_proj=pems_gdf_proj.drop(['DETECTOR_ID','LANE','DETECTOR_TYPE'],axis=1).reset_index()
pems_gdf_proj.head()

,index,STATION_ID,STATUS,STATION_TYPE,DISTRICT,CITY,FREEWAY,DIRECTION,LENGTH,STATE_POSTMILE,ABSOLUTE_POSTMILE,LATITUDE,LONGITUDE,PHYSICAL_LANES,COUNTY,geometry
0,0,818606,1,ML,8,NaN,215,NB,0.400,36.5,28.167,33.917895,-117.287828,3,riverside,POINT (250789.74 -451686.444)
1,1,402133,1,ML,4,70098.0,101,NB,0.275,20.87,489.572,38.447225,-122.724799,3,sonoma,POINT (-237497.887 51309.355)
2,2,407374,1,ML,4,49670.0,85,SB,0.495,R22.92,23.096,37.395123,-122.068612,3,santa clara,POINT (-182865.752 -67065.933)
3,3,763325,1,ML,7,44000.0,10,WB,0.290,R6.46,4.305,34.031891,-118.416029,4,los angeles,POINT (146268.916 -441407.596)
4,4,408911,1,ML,4,NaN,880,SB,0.570,.54,0.540,37.325153,-121.940859,4,santa clara,POINT (-171733.204 -75081.137)


In [44]:
# save the PeMS station
pems_gdf_proj.drop(['geometry'],axis=1).to_csv('PeMS_ML_HOV_Stations.csv')

In [45]:
pems_gdf_proj.shape

(12323, 16)

In [48]:
# Correct buffer sizes in meters and feet
buffer_sizes_meter = [9.144,12.192,15.24,18.288,21.336,24.384,30.48]
buffer_sizes_feet = [30,40,50,60,70,80,100]
# Store results
outside_pems_list = []

for i, buffer_m in enumerate(buffer_sizes_meter):
    # Create buffer around SHS
    shs_proj_link["geometry"] = shs_proj_link.buffer(buffer_m)
    
    # Find PeMS stations inside the buffer
    pems_inside = gpd.sjoin(pems_gdf_proj, shs_proj_link, predicate="intersects", how="inner")
    
    # Find PeMS stations outside the buffer
    pems_outside = pems_gdf_proj.loc[~pems_gdf_proj.index.isin(pems_inside.index)].copy()
    
    # Track buffer radius in feet
    pems_outside["buffer_radius_in_feet"] = buffer_sizes_feet[i]
    
    # Append to list
    outside_pems_list.append(pems_outside)

# Combine results
pems_outside_gdf = pd.concat(outside_pems_list, ignore_index=True)

pems_outside_gdf.head()


,index,STATION_ID,STATUS,STATION_TYPE,DISTRICT,CITY,FREEWAY,DIRECTION,LENGTH,STATE_POSTMILE,ABSOLUTE_POSTMILE,LATITUDE,LONGITUDE,PHYSICAL_LANES,COUNTY,geometry,buffer_radius_in_feet
0,22,1117909,1,ML,11,66000.0,15,NB,0.401,12.95,13.207,32.875159,-117.106891,6,san diego,POINT (271041.996 -566716.553),30
1,31,1119921,1,ML,11,11194.0,5,NB,0.791,46.545,46.436,33.115949,-117.318110,4,san diego,POINT (250504.864 -540633.306),30
2,39,10153210,1,ML,10,46898.0,99,NB,0.478,R12.13,185.476,37.278090,-120.444767,3,merced,POINT (-39381.506 -81978.216),30
3,57,763291,1,HV,7,44000.0,110,SB,0.400,15.88,15.810,33.958616,-118.281057,2,los angeles,POINT (158878.702 -449316.952),30
4,71,767839,1,HV,7,NaN,105,EB,0.510,R2.5,2.500,33.931513,-118.361927,1,los angeles,POINT (151456.737 -452454.981),30


In [49]:
pems_outside_gdf.groupby(['buffer_radius_in_feet'])['STATION_ID'].count()

buffer_radius_in_feet
30     886
40     213
50     130
60     100
70      81
80      70
100     56
Name: STATION_ID, dtype: int64

In [50]:
# Function to generate Google Maps URL
def generate_google_maps_url(row):
    return f"https://www.google.com/maps?q={row['LATITUDE']},{row['LONGITUDE']}"

# Apply the function to create a new column
pems_outside_gdf['Google_URL'] = pems_outside_gdf.apply(generate_google_maps_url, axis=1)
pems_outside_gdf=pems_outside_gdf.dropna(subset=['LATITUDE','LONGITUDE'])
pems_outside_gdf=pems_outside_gdf.drop(['geometry'],axis=1)
pems_outside_gdf.head()

,index,STATION_ID,STATUS,STATION_TYPE,DISTRICT,CITY,FREEWAY,DIRECTION,LENGTH,STATE_POSTMILE,ABSOLUTE_POSTMILE,LATITUDE,LONGITUDE,PHYSICAL_LANES,COUNTY,buffer_radius_in_feet,Google_URL
0,22,1117909,1,ML,11,66000.0,15,NB,0.401,12.95,13.207,32.875159,-117.106891,6,san diego,30,"https://www.google.com/maps?q=32.875159,-117.1..."
1,31,1119921,1,ML,11,11194.0,5,NB,0.791,46.545,46.436,33.115949,-117.318110,4,san diego,30,"https://www.google.com/maps?q=33.115949,-117.3..."
2,39,10153210,1,ML,10,46898.0,99,NB,0.478,R12.13,185.476,37.278090,-120.444767,3,merced,30,"https://www.google.com/maps?q=37.27809,-120.44..."
3,57,763291,1,HV,7,44000.0,110,SB,0.400,15.88,15.810,33.958616,-118.281057,2,los angeles,30,"https://www.google.com/maps?q=33.958616,-118.2..."
4,71,767839,1,HV,7,NaN,105,EB,0.510,R2.5,2.500,33.931513,-118.361927,1,los angeles,30,"https://www.google.com/maps?q=33.931513,-118.3..."


In [51]:
# save the data
pems_outside_gdf.to_csv("PeMS_sensor_outside_buffer.csv")

In [55]:
# now read the PeMS near distance from SHS link
pems_dist=pd.read_excel('pems_nears_dist.xls')
pems_dist['NEAR_DIST_ft']=pems_dist['NEAR_DIST']
pems_dist=pems_dist.drop(['OBJECTID','NEAR_FID','NEAR_DIST'],axis=1)
pems_dist.head()

,IN_FID,NEAR_DIST_ft
0,1,7.118809
1,2,3.663812
2,3,0.929097
3,4,5.282708
4,5,14.532024


In [56]:
pems_dist.IN_FID.nunique(),pems_gdf_proj11.IN_FID.nunique()

(12317, 12323)

In [57]:
pems_dist.shape, pems_gdf_proj.shape

((12317, 2), (12323, 16))

In [63]:
pems_gdf_proj11=pems_gdf_proj.copy().reset_index()
pems_gdf_proj11['IN_FID']=pems_gdf_proj11.index +1
pems_gdf_proj12=pd.merge(pems_gdf_proj11,pems_dist, on=['IN_FID'], how='inner' )
pems_gdf_proj12=pems_gdf_proj12.drop(['level_0','index','IN_FID'],axis=1)
pems_gdf_proj12.head()

,STATION_ID,STATUS,STATION_TYPE,DISTRICT,CITY,FREEWAY,DIRECTION,LENGTH,STATE_POSTMILE,ABSOLUTE_POSTMILE,LATITUDE,LONGITUDE,PHYSICAL_LANES,COUNTY,geometry,NEAR_DIST_ft
0,818606,1,ML,8,NaN,215,NB,0.400,36.5,28.167,33.917895,-117.287828,3,riverside,POINT (250789.74 -451686.444),7.118809
1,402133,1,ML,4,70098.0,101,NB,0.275,20.87,489.572,38.447225,-122.724799,3,sonoma,POINT (-237497.887 51309.355),3.663812
2,407374,1,ML,4,49670.0,85,SB,0.495,R22.92,23.096,37.395123,-122.068612,3,santa clara,POINT (-182865.752 -67065.933),0.929097
3,763325,1,ML,7,44000.0,10,WB,0.290,R6.46,4.305,34.031891,-118.416029,4,los angeles,POINT (146268.916 -441407.596),5.282708
4,408911,1,ML,4,NaN,880,SB,0.570,.54,0.540,37.325153,-121.940859,4,santa clara,POINT (-171733.204 -75081.137),14.532024


In [64]:
pems_gdf_proj12.shape

(12317, 16)

In [77]:
# define distance bin
def dist_bin (NEAR_DIST_ft):
    if NEAR_DIST_ft<30:
        return '<30 ft'
    elif (NEAR_DIST_ft<40 and NEAR_DIST_ft>=30):
        return '30-39 ft'
    elif (NEAR_DIST_ft<50 and NEAR_DIST_ft>=40):
        return '40-49 ft'
    elif (NEAR_DIST_ft<60 and NEAR_DIST_ft>=50):
        return '50-59 ft'
    elif (NEAR_DIST_ft<70 and NEAR_DIST_ft>=60):
        return '60-69 ft'
    elif (NEAR_DIST_ft<80 and NEAR_DIST_ft>=70):
        return '70-79 ft'
    elif (NEAR_DIST_ft<90 and NEAR_DIST_ft>=80):
        return '80-89 ft'
    elif (NEAR_DIST_ft<100 and NEAR_DIST_ft>=90):
        return '90-99 ft'
    else:
        return '>=100 ft'

In [78]:
pems_gdf_proj12['Air_Distance'] = pems_gdf_proj12.apply(lambda row: dist_bin(row['NEAR_DIST_ft']), axis=1)
pems_gdf_proj12.head()


,STATION_ID,STATUS,STATION_TYPE,DISTRICT,CITY,FREEWAY,DIRECTION,LENGTH,STATE_POSTMILE,ABSOLUTE_POSTMILE,LATITUDE,LONGITUDE,PHYSICAL_LANES,COUNTY,NEAR_DIST_ft,Air_Distance,Google_URL
0,818606,1,ML,8,NaN,215,NB,0.400,36.5,28.167,33.917895,-117.287828,3,riverside,7.118809,<30 ft,"https://www.google.com/maps?q=33.917895,-117.2..."
1,402133,1,ML,4,70098.0,101,NB,0.275,20.87,489.572,38.447225,-122.724799,3,sonoma,3.663812,<30 ft,"https://www.google.com/maps?q=38.447225,-122.7..."
2,407374,1,ML,4,49670.0,85,SB,0.495,R22.92,23.096,37.395123,-122.068612,3,santa clara,0.929097,<30 ft,"https://www.google.com/maps?q=37.395123,-122.0..."
3,763325,1,ML,7,44000.0,10,WB,0.290,R6.46,4.305,34.031891,-118.416029,4,los angeles,5.282708,<30 ft,"https://www.google.com/maps?q=34.031891,-118.4..."
4,408911,1,ML,4,NaN,880,SB,0.570,.54,0.540,37.325153,-121.940859,4,santa clara,14.532024,<30 ft,"https://www.google.com/maps?q=37.325153,-121.9..."


In [79]:
pems_gdf_proj12.groupby(['Air_Distance'])['Air_Distance'].count()

Air_Distance
30-39 ft      427
40-49 ft      149
50-59 ft       71
60-69 ft       26
70-79 ft       24
80-89 ft       23
90-99 ft       15
<30 ft      11437
>=100 ft      145
Name: Air_Distance, dtype: int64

In [80]:
# Apply the function to create a new column
pems_gdf_proj12['Google_URL'] = pems_gdf_proj12.apply(generate_google_maps_url, axis=1)
pems_gdf_proj12=pems_gdf_proj12.dropna(subset=['LATITUDE','LONGITUDE'])
#pems_gdf_proj12=pems_gdf_proj12.drop(['geometry'],axis=1)
pems_gdf_proj12.head()

,STATION_ID,STATUS,STATION_TYPE,DISTRICT,CITY,FREEWAY,DIRECTION,LENGTH,STATE_POSTMILE,ABSOLUTE_POSTMILE,LATITUDE,LONGITUDE,PHYSICAL_LANES,COUNTY,NEAR_DIST_ft,Air_Distance,Google_URL
0,818606,1,ML,8,NaN,215,NB,0.400,36.5,28.167,33.917895,-117.287828,3,riverside,7.118809,<30 ft,"https://www.google.com/maps?q=33.917895,-117.2..."
1,402133,1,ML,4,70098.0,101,NB,0.275,20.87,489.572,38.447225,-122.724799,3,sonoma,3.663812,<30 ft,"https://www.google.com/maps?q=38.447225,-122.7..."
2,407374,1,ML,4,49670.0,85,SB,0.495,R22.92,23.096,37.395123,-122.068612,3,santa clara,0.929097,<30 ft,"https://www.google.com/maps?q=37.395123,-122.0..."
3,763325,1,ML,7,44000.0,10,WB,0.290,R6.46,4.305,34.031891,-118.416029,4,los angeles,5.282708,<30 ft,"https://www.google.com/maps?q=34.031891,-118.4..."
4,408911,1,ML,4,NaN,880,SB,0.570,.54,0.540,37.325153,-121.940859,4,santa clara,14.532024,<30 ft,"https://www.google.com/maps?q=37.325153,-121.9..."


In [71]:
pems_gdf_proj12.shape

(12317, 17)

In [81]:
pems_gdf_proj12.to_csv('PeMS_sensor_distance_from_SHS_centerline.csv')

In [341]:
# end of the script